## Simple WPS Profile with optional func decorator

In [ ]:
from pywps import ComplexInput, ComplexOutput, FORMATS, Format
from pywps import Process

In [ ]:
esgf_api_test_a_input = ComplexInput(
        'test_a', 'Test A ESGF API Param',
         supported_formats=[FORMATS.JSON],
)

esgf_api_test_b_input = ComplexInput(
        'test_b', 'Test B ESGF API Param',
         supported_formats=[FORMATS.JSON],
)

esgf_api_inputs = [esgf_api_test_a_input, esgf_api_test_b_input]


def esgf_api(F):
    def wrapper(self):
        F(self)
        self.profile.append('ESGF-API')
        self.inputs.extend(esgf_api_inputs)
    return wrapper

### Use input parameters from esgf_api profile

In [ ]:
class EmuSubsetSimple(Process):
    def __init__(self):
        domain = ComplexInput(
                    'domain', 'domain',
                    abstract="",
                    supported_formats=[FORMATS.JSON],
                    min_occurs=0, max_occurs=1)
        inputs = [domain, ]
        # extend by esgf api profile
        inputs.extend(esgf_api_inputs)
        outputs = [
            ComplexOutput(
                'output', 'Output',
                as_reference=True,
                supported_formats=[FORMATS.NETCDF], ), ]
        
        super(EmuSubsetSimple, self).__init__(
            self._handler,
            identifier='Emu.subset',
            title='xarray.subset',
            abstract="subset netcdf files",
            version='1',
            inputs=inputs,
            outputs=outputs,
            store_supported=True,
            status_supported=True)
    
    def _handler(self, request, response):
        pass

In [ ]:
p = EmuSubsetSimple()
for inpt in p.inputs:
    print(inpt.title)

### Use esgf_api decorator

In [ ]:
class EmuSubsetDecorated(Process):
    # use esgf_api decorator to extend wps inputs behind the scences
    @esgf_api  
    def __init__(self):
        domain = ComplexInput(
                    'domain', 'domain',
                    abstract="",
                    supported_formats=[FORMATS.JSON],
                    min_occurs=0, max_occurs=1)
        inputs = [domain]
        outputs = [
            ComplexOutput(
                'output', 'Output',
                as_reference=True,
                supported_formats=[FORMATS.NETCDF], ), ]
        
        super(EmuSubsetDecorated, self).__init__(
            self._handler,
            identifier='Emu.subset',
            title='xarray.subset',
            abstract="subset netcdf files",
            version='1',
            inputs=inputs,
            outputs=outputs,
            store_supported=True,
            status_supported=True)
    
    def _handler(self, request, response):
        pass

In [ ]:
p = EmuSubsetDecorated()
for inpt in p.inputs:
    print(inpt.title)

In [ ]:
p.profile